# Ohms Law Lab

### ~~Submit your GitHub link for your plot of V vs I for the unknown resistor (4 pts)~~
### ~~What was the directly measured (with your multimeter) value for the resistance of the unknown resistor including the uncertainty.  This is considered to be the "accepted" value. (2 pts)~~

As stated in class, we didn't have to do this part of the lab this week, as we did a similar plot for last weeks lab.

Usual error code...

In [2]:
from IPython.display import Markdown
from typing import Iterable, Tuple, Union, Any
import numpy as np
# Define our rules...

def rule_1(c, error_val):
    return abs(c) * error_val

# res = c * val ^ power, where error in value is error_val....
def rule_2(c, val, error_val, power):
    return abs(c * power * val ** (power - 1)) * error_val

def rule_3(*err_vals: Iterable[float]) -> float:
    """
    Calculate rule 3 from the paper 'Treatment of Data'.
    
    @param err_vals: A list of parameters, being the errors in each value. It is assumed they were summed 
                     together to get the final value.
    
    @returns: The error of all of the sum of the values...
    """
    total = 0
    
    for err in err_vals:
        total += err ** 2
    
    return np.sqrt(total)


def rule_4(value: float, *error_list: Iterable[Tuple[float, float, float]]) -> float:
    """
    Calculate rule 4 from the paper 'Treatment of Data'.
    
    @param value: The value of the thing we are trying to calculate the error of.
    @param error_list: A list of length 3 tuples. 
                       Each tuple should contain:
                       - A float: A value in the error formula.
                       - A float: The measured error in the above value.
                       - A float: The power of the above value in the multiplicative formula.

    @returns: A float, being the error in 'value'.
    """
    total = 0
    
    for x, x_err, power in error_list:
        total += (power * (x_err / x)) ** 2
        
    return abs(value) * np.sqrt(total)

FloatVec = Union[float, np.ndarray] 
BoolVec = Union[bool, np.ndarray]
OpVec = Union[Any, Iterable]
StrVec = Union[str, Iterable[str]]

def values_agree(val_1: FloatVec, err_1: FloatVec, val_2: FloatVec, err_2: FloatVec) -> BoolVec:
    """
    Determines if the values in 2 vectors(arrays) agree with each other, given there uncertainty values. 
    
    @param val_1: The 1st array of values.
    @param err_1: The uncertainty values for the 1st vector.
    @param val_2: The 2nd array of values.
    @param err_2: The uncertainty values for the 2st vector. 
    
    @returns: A vector of booleans, being whether each value agrees with the other.
    """
    # Grab the ranges for each value...
    r11, r12 = val_1 - err_1, val_1 + err_1
    r21, r22 = val_2 - err_2, val_2 + err_2
    
    # The ranges are sorted (r21 <= r22 and r11 <= r12), so the simple 2 checks below are enough. 
    # Check 1: Does r21(lowest value of 2nd range) land above the 1st range? If so fail...
    # Check 2: Does r22(highest value of 2nd range) fall below the 1st range? If so fail...
    return ((r21 <= r12) & (r11 <= r22))

# Some extra stuff for pretty printing measurements....
from IPython.display import Markdown

def format_result(msgs: OpVec, value: OpVec, value_error: OpVec, units: OpVec, past_dec: OpVec = 2) -> StrVec:
    all_arrs = [msgs, value, value_error, units, past_dec]
    
    def to_vec(val, types): 
        return [val] if(isinstance(val, types)) else list(val)
    
    (msgs, value, value_error, units, past_dec) = all_arrs = [
        to_vec(v, (float, int, str)) for v in all_arrs
    ]
    max_len = max(len(arr) for arr in all_arrs)
    
    msgs, value, value_error, units, past_dec = all_arrs = [
        (v * max_len if(len(v) == 1) else v) for v in all_arrs
    ]
    
    # Nesting variable in the formating of variables, not confusing at all....
    return [
        fr"{msg}$ {v:.0{pd}f} \pm {v_err:.0{pd}f} \text{{ }} {u} $"
        for msg, v, v_err, pd, u in zip(msgs, value, value_error, past_dec, units)
    ]

def display_result(*args, **kwargs):
    for res in format_result(*args, **kwargs):
        display(Markdown(res))

# Print function for markdown :)...
def mkdwn(string):
    display(Markdown(string))

## Measurements

Here is the entire table of values that I measured, and used for my calculations.

In [3]:
import pandas as pd

# Load in the raw data for both parts...
i_and_v = pd.read_csv("i_and_v.csv", index_col=0)
display(i_and_v)

,V1 (V),V2 (V),I1 (mA),I2 (mA),VT (V),V Error (V),I Error (mA)
Circut,,,,,,,
Series,2.55,0.48,26.0,26.0,3.17,0.01275,0.3120
Parallel,2.91,2.91,29.2,138.6,0.00,0.01455,1.6632


### 1.) Write down your measurements of the potential drops V1, V2 and Vtotal for the circuit with the two resistors in series with uncertainties. (3 pts)

The following code below loads in the dataset, computes the voltage errors, and displays the voltages and there uncertainties. See the output of the cell below to get the values for V1, V2, and Vtotal... 

In [4]:
import pandas as pd

# Load in the raw data for both parts...
i_and_v = pd.read_csv("i_and_v.csv", index_col=0)
V1 = i_and_v["V1 (V)"]
V2 = i_and_v["V2 (V)"]
err_V = i_and_v["V Error (V)"]

V_total = i_and_v["VT (V)"]["Series"]
err_V_total = rule_3(err_V, err_V)["Series"]

# Display the results using our new display function....
display_result(
    ["$V_1$: ", "$V_2$: ", "$V_{total}$: "], 
    [V1["Series"], V2["Series"], V_total], 
    [err_V["Series"], err_V["Series"], err_V_total], 
    "V"
)

$V_1$: $ 2.55 \pm 0.01 \text{ } V $

$V_2$: $ 0.48 \pm 0.01 \text{ } V $

$V_{total}$: $ 3.17 \pm 0.02 \text{ } V $

### 2.) Write down your measurements of the currents I1, I2, and Itotal for the circuit with two resistors in parallel.     (3 pts)

Listed in the output for the below code cell are my measurements of I1, I2, and Itotal for the parallel circuit.

In [5]:
I1 = i_and_v["I1 (mA)"] / 1000  # mA -> A 
I2 = i_and_v["I2 (mA)"] / 1000  # mA -> A
err_I = i_and_v["I Error (mA)"] / 1000  # mA -> A

sum_I = (I1 + I2)[1]
err_sum_I = rule_3(err_I, err_I)["Parallel"]

# Quickly convert back to mA for display, otherwise values are really small...
display_result(
    ["$I_1$: ", "$I_2$: ", "$I_{total}$: "], 
    [i * 1000 for i in [I1["Parallel"], I2["Parallel"], sum_I]], 
    [i * 1000 for i in [err_I["Parallel"], err_I["Parallel"], err_sum_I]], 
    "mA"
)

$I_1$: $ 29.20 \pm 1.66 \text{ } mA $

$I_2$: $ 138.60 \pm 1.66 \text{ } mA $

$I_{total}$: $ 167.80 \pm 2.35 \text{ } mA $

## Calculations

### ~~What is the value of the resistance that you found from your plot for the single resistor circuit including the uncertainty?  Does this resistance agree with the one that you directly measured using the multimeter?  Justify your answer.  (3 points)~~

Again, did not do this part of the experiment.

### What was the value of the equivalent resistance that you calculated for the circuit with two resistors in series from your measurements of Vtotal and Itotal. Make sure to include the estimated uncertainty.  Does this value agree with the directly measured (with the ohmmeter) equivalent resistance value?  Justify your answer. (4 points)

See the code block below for code calculations. For the first part of the experiment, I got a total resistance of $ 121.92 \pm 1.62 \text{ } \Omega $. This value does not agree with the theoretical value a calculated, which is $ 117.70 \pm 0.80 \text{ } \Omega $, as the error ranges on theses values don't overlap at any given location. I suspect this is due to the errors given in the operators manual for measurements not being enough, as I suspect more error in the instrument. Also, the meter seems to consistently read resistance values significantly less than both the experimental values and labeled values on the resistor (also did this during the last exercise), so I suspect there is an issue with my multimeter.

### What was the value of the equivalent resistance that you calculated for the circuit with two resistors in parallel from your measurements. Make sure to include the estimated uncertainty.  Does this value agree with the directly measured (with the ohmmeter) equivalent resistance value?  Justify your answer. (4 points)

See the code block below for code calculations. For the second part of the experiment, I got a total resistance of $ 17.34 \pm 0.26 \text{ } \Omega $. This again, does not agree with my directly measured values for the total resistance, which gave a result of $ 16.34 \pm 0.11 \text{ } \Omega $, which does not overlap the experimental result at any given location when comparing their error ranges. I suspect the values failed to agree for similar reasons as the first part, specifically the multimeter giving poor direct resistance measurements.

In [9]:
r = pd.read_csv("r.csv")
# Compute experimental R1 for both parts....
R1 = V1 / I1
err_R1 = rule_4(R1, (I1, err_I, -1), (V1, err_V, 1))
mkdwn("#### Experimental $R_1$: ")
display_result(["$R_{1}$ Series: ", "$R_{1}$ Parallel: "], R1, err_R1, "\Omega")

# Compute experimental R2 for both parts...
R2 = V2 / I2
err_R2 = rule_4(R2, (I2, err_I, -1), (V2, err_V, 1))
mkdwn("#### Experimental $R_2$: ")
display_result(["$R_{2}$ Series: ", "$R_{2}$ Parallel: "], R2, err_R2, "\Omega")

# R total for part one of the experiment.
R_total_p1 = (V_total / I1[0])
err_R_total_p1 = rule_4(R_total_p1, (V_total, err_V_total, 1), (I1[0], err_I[0], -1))
mkdwn("#### Experimental $R_{total}$ for Series:")
display_result("$R_{total}$ Parallel: ", R_total_p1, err_R_total_p1, "\Omega")

# R total for part two of the experiment...
R_total_p2 = (V1[1] / sum_I)
err_R_total_p2 = rule_4(R_total_p2, (V1[1], err_V[1], 1), (sum_I, err_sum_I, -1))
mkdwn("#### Experimental $R_{total}$ for Parallel:")
display_result("$R_{total}$ Series: ", R_total_p2, err_R_total_p2, "\Omega")

# R1 and R2 measured directly....
R1_direct, R2_direct = r["R Measured (Ohms)"]
err_R1_direct, err_R2_direct = r["R Error (Ohms)"]
mkdwn("#### Directly Measured $R$ Values: ")
display_result(["$R_1$: ", "$R_2$: "], [R1_direct, R2_direct], [err_R1_direct, err_R2_direct], "\Omega")

R_total_p1_direct = R1_direct + R2_direct
err_R_total_p1_direct = rule_3(err_R1_direct, err_R2_direct)
mkdwn("#### Directly Calculated(Theoretical) $R_{total}$ for Series:")
display_result("$R_{total}$ Parallel: ", R_total_p1_direct, err_R_total_p1_direct, "\Omega")

inv = (1 / R1_direct) + (1 / R2_direct)
R_total_p2_direct = 1 / inv
err_inv = rule_3(rule_2(1, R1_direct, err_R1_direct, -1), rule_2(1, R2_direct, err_R2_direct, -1))
err_R_total_p2_direct = rule_2(1, inv, err_inv, -1)
mkdwn("#### Directly Calculated(Theoretical) $R_{total}$ for Paralell:")
display_result("$R_{total}$ Parallel: ", R_total_p2_direct, err_R_total_p2_direct, "\Omega")

#### Experimental $R_1$: 

$R_{1}$ Series: $ 98.08 \pm 1.28 \text{ } \Omega $

$R_{1}$ Parallel: $ 99.66 \pm 5.70 \text{ } \Omega $

#### Experimental $R_2$: 

$R_{2}$ Series: $ 18.46 \pm 0.54 \text{ } \Omega $

...

$R_2$: $ 19.60 \pm 0.16 \text{ } \Omega $

#### Directly Calculated(Theoretical) $R_{total}$ for Series:

$R_{total}$ Parallel: $ 117.70 \pm 0.80 \text{ } \Omega $

#### Directly Calculated(Theoretical) $R_{total}$ for Paralell:

$R_{total}$ Parallel: $ 16.34 \pm 0.11 \text{ } \Omega $

## Concepts

### 1.) What should have been the relationship between the currents I1, I2 and Itotal for the circuit with two resistors in parallel? (2 points)

We see the relationship for resistance is:
$$
\frac {1} {R_f} = \frac {1} {R_1} + \frac {1} {R_2}
$$
Now, plugging in $\frac{V}{I}$ for $R$, and noting $V_1=V_2=V_f$ we can solve for $I_f$:
$$
\frac {I_f}{V} = \frac {I_1}{V} + \frac {I_2}{V} \implies I_f = I_1 + I_2
$$

Notice, this tells us the total or final current equals the sum of the two currents passing through each resistor. This makes sense intrinsically also, as the current gets divided between the two paths or resistors, yet we know that none of the current 'disappears', as we can see the same total current coming out the other side.

### 1.) If you were to add another 100 ohm resistor to the parallel circuit, would the total resistance increase or decrease? (2 points)

If we add another resistor to the circuit, we provide another pathway for the current to travel down, and therefore more current can travel through our resistor array. By definition, since voltage is constant and the current has increased, the resistance resisting the current must have gone down to account for this. This intuitive conclusion matches the conclusion we would arrive at by applying the formula for calculating the resistance of parallel resistors, as we add another reciprocal term.